In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

# Chapter 8. Join

## 8.2 조인 타입
+ 스파크의 조인 타입
    + inner join, outer join, left outer join, right outer join
    + left semi join: 왼쪽 데이터셋의 키가 오른쪽 데이터셋에 있는 경우에는 키가 일치하는 왼쪽 데이터셋만 유지 
    + left anti join: 왼쪽 데이터셋의 키가 오른쪽 데이터셋에 없는 경우에는 키가 일치하지 않는 왼쪽 데이터셋만 유지
    + natural join: 두 데이터셋에서 동일한 이름을 가진 컬럼을 암시적으로 결합하는 조인을 수행
    + cross join: 왼쪽 데이터셋의 모든 로우와 오른쪽 데이터 셋의 모든 로우를 조합


In [2]:
""" 예제에서 사용할 데이터 셋 """
person = spark.createDataFrame([
    (0, "Bill Chambers", 0, [100]),
    (1, "Matei Zaharia", 1, [500, 250, 100]),
    (2, "Michael Arnbrust", 1, [250, 100])])\
.toDF("id", "name", "graduate_program", "spark_status")

graduateProgram = spark.createDataFrame([
    (0, "Masters", "School of Information", "UC Berkeley"),
    (2, "Masters", "EECS", "UC Berkeley"),
    (1, "Ph. D", "EECS", "UC Berkeley")])\
.toDF("id", "degree", "department", "school")

sparkStatus = spark.createDataFrame([
    (500, "Vice President"),
    (250, "PMC Member"),
    (100, "Contributor")])\
.toDF("id", "status")

In [3]:
person.createOrReplaceTempView("person")
graduateProgram.createOrReplaceTempView("graduateProgram")
sparkStatus.createOrReplaceTempView("sparkStatus")

In [4]:
print("person")
person.show()
print("graduateProgram")
graduateProgram.show()
print("sparkStatus")
sparkStatus.show()

person
+---+----------------+----------------+---------------+
| id|            name|graduate_program|   spark_status|
+---+----------------+----------------+---------------+
|  0|   Bill Chambers|               0|          [100]|
|  1|   Matei Zaharia|               1|[500, 250, 100]|
|  2|Michael Arnbrust|               1|     [250, 100]|
+---+----------------+----------------+---------------+

graduateProgram
+---+-------+--------------------+-----------+
| id| degree|          department|     school|
+---+-------+--------------------+-----------+
|  0|Masters|School of Informa...|UC Berkeley|
|  2|Masters|                EECS|UC Berkeley|
|  1|  Ph. D|                EECS|UC Berkeley|
+---+-------+--------------------+-----------+

sparkStatus
+---+--------------+
| id|        status|
+---+--------------+
|500|Vice President|
|250|    PMC Member|
|100|   Contributor|
+---+--------------+



## 8.3 내부 조인
+ 참으로 평가되는 로우만 결합
+ 세 번째 파라미터로 조인 타입을 명확하게 지정할 수 있음

In [5]:
joinExpression = person["graduate_program"] == graduateProgram["id"]
person.join(graduateProgram, joinExpression).show()

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph. D|                EECS|UC Berkeley|
|  2|Michael Arnbrust|               1|     [250, 100]|  1|  Ph. D|                EECS|UC Berkeley|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+



In [6]:
joinExpression = person["graduate_program"] == graduateProgram["id"]
person.join(graduateProgram, joinExpression, "inner").show()

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph. D|                EECS|UC Berkeley|
|  2|Michael Arnbrust|               1|     [250, 100]|  1|  Ph. D|                EECS|UC Berkeley|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+



## 8.4 외부 조인
+ 왼쪽이나 오른쪽 DataFrame에 일치하는 로우가 없다면 해당 위치에 null을 삽입

In [7]:
joinExpression = person["graduate_program"] == graduateProgram["id"]
person.join(graduateProgram, joinExpression, "outer").show() # "outer"는 외부 조인

+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|   0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|   1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph. D|                EECS|UC Berkeley|
|   2|Michael Arnbrust|               1|     [250, 100]|  1|  Ph. D|                EECS|UC Berkeley|
|null|            null|            null|           null|  2|Masters|                EECS|UC Berkeley|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+



## 8.5 왼쪽 외부 조인

In [8]:
person.join(graduateProgram, joinExpression, "left_outer").show() # "outer"는 외부 조인

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph. D|                EECS|UC Berkeley|
|  2|Michael Arnbrust|               1|     [250, 100]|  1|  Ph. D|                EECS|UC Berkeley|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+



## 8.6 오른쪽 외부 조인

In [9]:
person.join(graduateProgram, joinExpression, "right_outer").show() # "outer"는 외부 조인

+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|   0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|   1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph. D|                EECS|UC Berkeley|
|   2|Michael Arnbrust|               1|     [250, 100]|  1|  Ph. D|                EECS|UC Berkeley|
|null|            null|            null|           null|  2|Masters|                EECS|UC Berkeley|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+



## 8.7 왼쪽 세미 조인
+ 값이 존재하는 지 확인 용도, 값이 있다면 왼쪽 DataFrame에 중복 키가 있더라도 해당 로우는 결과에 포함
+ 기존 조인 기능과는 달리 DataFrame의 필터 기능과 유사

In [10]:
joinType = 'left_semi'
graduateProgram.join(person, joinExpression, joinType).show(5)

+---+-------+--------------------+-----------+
| id| degree|          department|     school|
+---+-------+--------------------+-----------+
|  0|Masters|School of Informa...|UC Berkeley|
|  1|  Ph. D|                EECS|UC Berkeley|
+---+-------+--------------------+-----------+



In [11]:
""" 중복 키의 처리 """
gradProgram2 = graduateProgram.union(spark.createDataFrame([
    (0, 'Masters', "Duplicated Row", "Duplicated School")
]))
gradProgram2.createOrReplaceTempView("gradProgram2")
gradProgram2.join(person, joinExpression, joinType).show(5)

+---+-------+--------------------+-----------------+
| id| degree|          department|           school|
+---+-------+--------------------+-----------------+
|  0|Masters|School of Informa...|      UC Berkeley|
|  0|Masters|      Duplicated Row|Duplicated School|
|  1|  Ph. D|                EECS|      UC Berkeley|
+---+-------+--------------------+-----------------+



## 8.8 왼쪽 안티 조인
+ 왼쪽 세미 조인의 반대 개념, 즉 오른쪽 DataFrame의 어떤 값도 포함하지 않음
+ SQL의 NOT IN과 같은 스타일의 필터

In [12]:
joinType = 'left_anti'
graduateProgram.join(person, joinExpression, joinType).show()

+---+-------+----------+-----------+
| id| degree|department|     school|
+---+-------+----------+-----------+
|  2|Masters|      EECS|UC Berkeley|
+---+-------+----------+-----------+



## 8.9 자연 조인
+ 조인하려는 컬럼을 암시적으로 추정
+ 암시적인 처리는 언제나 위험하므로 비추천
+ Python join 함수는 이 기능을 지원하지 않음
```
-- SQL
SELECT * FROM graduateProgram NATURAL JOIN person
```

In [13]:
spark.sql("SELECT * FROM graduateProgram NATURAL JOIN person").show()

+---+-------+--------------------+-----------+----------------+----------------+---------------+
| id| degree|          department|     school|            name|graduate_program|   spark_status|
+---+-------+--------------------+-----------+----------------+----------------+---------------+
|  0|Masters|School of Informa...|UC Berkeley|   Bill Chambers|               0|          [100]|
|  1|  Ph. D|                EECS|UC Berkeley|   Matei Zaharia|               1|[500, 250, 100]|
|  2|Masters|                EECS|UC Berkeley|Michael Arnbrust|               1|     [250, 100]|
+---+-------+--------------------+-----------+----------------+----------------+---------------+



## 8.10 교차 조인(카테시안 조인)
+ 교차 조인은 조건절을 기술하지 않은 내부 조인을 의미
+ 왼쪽의 모든 로우를 오른쪽의 모든 로우와 결합함(결과의 로우 수 = 왼쪽 로우 수 * 오른쪽 로우 수)

In [14]:
""" 크로스 조인이지만 조건을 설정해야 하며, 조건에 부합된 결과를 출력하여 inner조인과 동일 """
joinType = "cross"
graduateProgram.join(person, on=joinExpression, how=joinType).show()

+---+-------+--------------------+-----------+---+----------------+----------------+---------------+
| id| degree|          department|     school| id|            name|graduate_program|   spark_status|
+---+-------+--------------------+-----------+---+----------------+----------------+---------------+
|  0|Masters|School of Informa...|UC Berkeley|  0|   Bill Chambers|               0|          [100]|
|  1|  Ph. D|                EECS|UC Berkeley|  1|   Matei Zaharia|               1|[500, 250, 100]|
|  1|  Ph. D|                EECS|UC Berkeley|  2|Michael Arnbrust|               1|     [250, 100]|
+---+-------+--------------------+-----------+---+----------------+----------------+---------------+



In [15]:
person.crossJoin(graduateProgram).show()

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|  0|   Bill Chambers|               0|          [100]|  2|Masters|                EECS|UC Berkeley|
|  0|   Bill Chambers|               0|          [100]|  1|  Ph. D|                EECS|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  0|Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  2|Masters|                EECS|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph. D|                EECS|UC Berkeley|
|  2|Michael Arnbrust|               1|     [250, 100]|  0|Masters|School of Informa...|UC 

## 8.11 조인 사용 시 문제점

### 8.11.1 복합 데이터 타입의 조인
+ 불리언을 반환하는 모든 표현식은 조인 표현식으로 간주할 수 있음

In [16]:
from pyspark.sql.functions import expr

person.withColumnRenamed("id", "personId")\
    .join(sparkStatus, expr("array_contains(spark_status, id)")).show()

+--------+----------------+----------------+---------------+---+--------------+
|personId|            name|graduate_program|   spark_status| id|        status|
+--------+----------------+----------------+---------------+---+--------------+
|       0|   Bill Chambers|               0|          [100]|100|   Contributor|
|       1|   Matei Zaharia|               1|[500, 250, 100]|500|Vice President|
|       1|   Matei Zaharia|               1|[500, 250, 100]|250|    PMC Member|
|       1|   Matei Zaharia|               1|[500, 250, 100]|100|   Contributor|
|       2|Michael Arnbrust|               1|     [250, 100]|250|    PMC Member|
|       2|Michael Arnbrust|               1|     [250, 100]|100|   Contributor|
+--------+----------------+----------------+---------------+---+--------------+



### 8.11.2 중복 컬럼명 처리


In [17]:
""" 잘못된 데이터셋(컬럼명 중복) """
gradProgramDupe = graduateProgram.withColumnRenamed("id", "graduate_program")
joinExpr = gradProgramDupe["graduate_program"] == person["graduate_program"]

In [18]:
gradProgramDupe.join(person, joinExpr, "inner").show() # 조인을 수행했음에도 두 개의 graduate_program 컬럼이 존재

+----------------+-------+--------------------+-----------+---+----------------+----------------+---------------+
|graduate_program| degree|          department|     school| id|            name|graduate_program|   spark_status|
+----------------+-------+--------------------+-----------+---+----------------+----------------+---------------+
|               0|Masters|School of Informa...|UC Berkeley|  0|   Bill Chambers|               0|          [100]|
|               1|  Ph. D|                EECS|UC Berkeley|  1|   Matei Zaharia|               1|[500, 250, 100]|
|               1|  Ph. D|                EECS|UC Berkeley|  2|Michael Arnbrust|               1|     [250, 100]|
+----------------+-------+--------------------+-----------+---+----------------+----------------+---------------+



In [20]:
""" 중복 컬럼 조회 시 오류 발생 """
gradProgramDupe.join(person, joinExpr, "inner").select("graduate_program").show()

AnalysisException: "Reference 'graduate_program' is ambiguous, could be: graduate_program, graduate_program.;"

In [21]:
""" 해결 방법 1: 다른 조인 표현식 사용 """
# 중복된 두 컬럼 중 하나가 자동 제거 됨
person.join(gradProgramDupe, "graduate_program").select("graduate_program").show()

+----------------+
|graduate_program|
+----------------+
|               0|
|               1|
|               1|
+----------------+



In [22]:
""" 해결 방법 2: 조인 후 컬럼 제거 """
gradProgramDupe.join(person, joinExpr).drop(gradProgramDupe["graduate_program"])\
    .select("graduate_program").show() # 조인을 수행했음에도 두 개의 graduate_program 컬럼이 존재

+----------------+
|graduate_program|
+----------------+
|               0|
|               1|
|               1|
+----------------+



In [23]:
""" 해결 방법 3: 조인 전 컬럼명 변경(가장 확실한 방법) """
fixed_gradProgram = gradProgramDupe.withColumnRenamed("graduate_program", "grad_id")
fixed_gradProgram.join(person, fixed_gradProgram["grad_id"] == person["graduate_program"]).show()

+-------+-------+--------------------+-----------+---+----------------+----------------+---------------+
|grad_id| degree|          department|     school| id|            name|graduate_program|   spark_status|
+-------+-------+--------------------+-----------+---+----------------+----------------+---------------+
|      0|Masters|School of Informa...|UC Berkeley|  0|   Bill Chambers|               0|          [100]|
|      1|  Ph. D|                EECS|UC Berkeley|  1|   Matei Zaharia|               1|[500, 250, 100]|
|      1|  Ph. D|                EECS|UC Berkeley|  2|Michael Arnbrust|               1|     [250, 100]|
+-------+-------+--------------------+-----------+---+----------------+----------------+---------------+



## 8.12 스파크의 조인 수행 방식

### 8.12.1 네트워크 통신 전략
+ 셔플조인: 전체 노드 간 통신을 유발
+ 브로드캐스트 조인: 최초 테이블을 전체 노드로 복제 후 통신없이 진행

#### 큰 테이블과 큰 테이블 조인
+ 전체 노드 간 통신이 발생하는 셔플 조인이 발생됨

#### 큰 테이블과 작은 테이블 조인
+ 작은 DataFrame을 클러스터 전체 워커에 복제한 후 통신없이 진행
+ 모든 단일 노드에서 개별적으로 조인이 수행되므로 CPU가 가장 큰 병목 구간이 됨
+ broadcast 함수를 통해 브로드캐스트 조인을 설정할 수 있으나 강제할 수는 없음(옵티마이저가 무시 가능)

In [25]:
from pyspark.sql.functions import broadcast
person.join(broadcast(gradProgramDupe), "graduate_program").select("graduate_program").show()

+----------------+
|graduate_program|
+----------------+
|               0|
|               1|
|               1|
+----------------+



#### 작은 테이블과 작은 테이블 조인
+ 스파크가 결정하도록 내버려두는 것이 제일 좋은 선택